In [2]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import sys, os, re, glob, parse
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_context('notebook')
sns.set_style('white')

ligan_dir = '/home/mtr22/bridges/pylon5/mc5phpp/mtragoza/liGAN'
sys.path.append(ligan_dir)

#from params import ParamSpace
#from results import aggregate_data, add_group_column, plot_strips, plot_dist, plot_corr

work_dir = os.path.join(ligan_dir, 'train_pdbbind')
os.chdir(work_dir)
os.getcwd()

'/home/mtr22/bridges/pylon5/mc5phpp/mtragoza/liGAN/train_pdbbind'

In [3]:
def read_err_file(err_file):
    error = None
    with open(err_file) as f:
        for line in f:
            if re.match(r'.*(Error|Exception|error|fault|failed).*', line):
                error = line.rstrip()
    return error

indent = '   '
metric_files = []
for expt_name in ['loss_types']:
    print(expt_name)
    
    for job_name in os.listdir(expt_name):
        print(indent + job_name)
        
        err_files = []
        for err_file in os.listdir(expt_name + '/' + job_name):
            
            m = re.match(r'slurm-(\d+)\.err', err_file)
            if not m:
                continue
                
            job_id = int(m.group(1))
            err_files.append((job_id, err_file))
            
        err_files = sorted(err_files)
            
        for job_id, err_file in err_files:
            error = read_err_file('/'.join([expt_name, job_name, err_file]))
            print(2*indent + err_file + indent + str(error)[:88])
        
        job_id = err_files[-1][0]
        metric_file = glob.glob('/'.join([expt_name, job_name, str(job_id), '*.training_output']))[0]
        metric_files.append(metric_file)
        
        print()

len(metric_files)

loss_types
   gen_a_disc_w_0
      slurm-7918323.err   F0227 02:11:38.217861 15979 filler.hpp:149] Check failed: blob->count()
      slurm-7928188.err   ValueError: could not broadcast input array from shape (50,16,48,48,48) into shape (16,1
      slurm-7928279.err   None
      slurm-7928314.err   slurmstepd: error: *** JOB 7928314 ON gpu049 CANCELLED AT 2020-02-28T03:46:05 DUE TO TIM

   gen_e_disc_x_0
      slurm-7918326.err   F0227 02:14:27.740367 25613 filler.hpp:149] Check failed: blob->count()
      slurm-7928191.err   ValueError: could not broadcast input array from shape (50,16,48,48,48) into shape (16,1
      slurm-7928282.err   None
      slurm-7928317.err   slurmstepd: error: *** JOB 7928317 ON gpu054 CANCELLED AT 2020-02-28T03:46:05 DUE TO TIM

   gen_a_disc_x_0
      slurm-7918324.err   F0227 02:14:20.005477 16612 filler.hpp:149] Check failed: blob->count()
      slurm-7928189.err   ValueError: could not broadcast input array from shape (50,16,48,48,48) into shape (16,1
  

4

In [7]:
name_format = 'gen_{gen_loss_type}_disc_{disc_loss_type}_{random_seed}'

job_dfs = []
for metric_file in metric_files:
    try:
        job_df = pd.read_csv(metric_file, sep=' ')
    except Exception as e:
        print(metric_file, e, file=sys.stderr)
        continue
    
    expt_name, job_name, job_id, _ = metric_file.split('/')
    out_file = '/'.join([expt_name, job_name, 'slurm-{}.out'.format(job_id)])
    err_file = '/'.join([expt_name, job_name, 'slurm-{}.err'.format(job_id)])
    
    job_df['expt_name'] = expt_name
    job_df['job_name'] = job_name
    job_df['job_id'] = int(job_id)
    job_df['out_file'] = out_file
    job_df['err_file'] = err_file
    
    # get params from job_name
    job_params = parse.parse(name_format, job_name).named
    for param, value in job_params.items():
        job_df[param] = value
    
    # get error from err_file
    job_df['job_error'] = read_err_file(err_file)
    
    job_dfs.append(job_df)

df = pd.concat(job_dfs)

In [9]:
df.set_index(['expt_name', 'job_name', 'job_id', 'iteration', 'test_data'])

disc_wass_loss  \
expt_name  job_name       job_id  iteration test_data                   
loss_types gen_a_disc_w_0 7928314 0         train           -0.002786   
                                            test            -0.001210   
                                  100       train           -0.027468   
                                            test            -0.028523   
                                  200       train           -0.184905   
...                                                               ...   
           gen_e_disc_w_0 7928316 10700     test            -3.315659   
                                  10800     train           -1.048557   
                                            test            -3.577954   
                                  10900     train           -4.031207   
                                            test           -10.351663   

                                                       disc_iter  \
expt_name  job_name       job_id  iteration test_data              
loss_types gen_a_disc_w_0 7928314 0         train            0.0   
                                            test             0.0   
                                  100       train          100.0   
                                            test           100.0   
                                  200       train          200.0   
...                                                          ...   
           gen_e_disc_w_0 7928316 10700     test         10700.0   
                                  10800     train        10800.0   
                                            test         10800.0   
                                  10900     train        10900.0   
                                            test         10900.0   

                                                       disc_grad_norm  \
expt_name  job_name       job_id  iteration test_data                   
loss_types gen_a_disc_w_0 7928314 0         train                 1.0   
                                            test                  1.0   
                                  100       train                 1.0   
                                            test                  1.0   
                                  200       train                 1.0   
...                                                               ...   
           gen_e_disc_w_0 7928316 10700     test                  1.0   
                                  10800     train                 1.0   
                                            test                  1.0   
                                  10900     train                 1.0   
                                            test                  1.0   

                                                       gen_kldiv_loss  \
expt_name  job_name       job_id  iteration test_data                   
loss_types gen_a_disc_w_0 7928314 0         train            0.002050   
                                            test             0.002806   
                                  100       train        13308.836426   
                                            test         12606.152344   
                                  200       train         3047.602002   
...                                                               ...   
           gen_e_disc_w_0 7928316 10700     test          1356.305646   
                                  10800     train         1163.835278   
                                            test          1164.123785   
                                  10900     train         1048.549115   
                                            test          1181.764679   

                                                         gen_L1_loss  \
expt_name  job_name       job_id  iteration test_data                  
loss_types gen_a_disc_w_0 7928314 0         train      444543.696875   
                                            test       447089.134375   
                                